In [141]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from torch import optim, nn
import torch

from plotting import plot_model_outputs

import beer

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [177]:
mean = np.array([3., 2.])
cov = np.array([[2., 0.], [0., .2]])
data = np.random.multivariate_normal(mean, cov, size=100)
data[:, 1] = data[:, 1] + (data[:, 0]-mean[0])**2

mean = data.mean(axis=0)
var = data.var(axis=0)

data = (data - mean) / np.sqrt(var)

fig = figure(
    title='Non-Linear subspace',
    width=400,
    height=400,
)
fig.circle(data[:, 0], data[:, 1])
#x = np.linspace(-1, 7, 1000)
#fig.line(x, (x-3)**2+2, color='red')
show(fig)

In [183]:
obs_dim = 2
latent_dim = 2
nb_samples = 10

enc_struct = nn.Sequential(
    nn.Linear(obs_dim, 10),
    nn.Tanh(),
    nn.Linear(10, 10),
    nn.Tanh()
)
encoder = beer.models.MLPNormalDiag(enc_struct, 10, obs_dim)

dec_struct = nn.Sequential(
    nn.Linear(latent_dim, 10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU()
)
decoder = beer.models.MLPNormalDiag(dec_struct, 10, obs_dim)

latent_model = beer.models.NormalDiagonalCovariance.create(latent_dim)
#latent_model = beer.models.NaturalIsotropicGaussian(2)

model = beer.models.VAE(encoder, decoder, latent_model, nb_samples)
model.encoder.hid_to_logvar.bias = nn.Parameter(torch.ones(2) * -.5)
model.decoder.hid_to_logvar.bias = nn.Parameter(torch.ones(1) * -.5)

In [184]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-2)
history = beer.inference.History(report_interval=250)

In [188]:
model.sample = False
beer.inference.run_training(data, model, optimizer, 1000, history, batch_size=20, lrate_latent_model=1.0, kl_weight=0.0)

Epoch: 1250 	elbo: 21.688892 llh: 433.777832 kld: 0.000000
Epoch: 1500 	elbo: 23.159624 llh: 463.192474 kld: 0.000000
Epoch: 1750 	elbo: 22.802872 llh: 456.057434 kld: 0.000000
Epoch: 2000 	elbo: 19.463403 llh: 389.268055 kld: 0.000000


In [189]:
plot_model_outputs(model, data[:100])

In [113]:
history.plot()

In [152]:
exp = model.latent_model.posterior.grad_lognorm()
print(1/ (exp[0] * -2))

[ 1.  1.]
